In [1]:
import sys
sys.executable
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages')

In [2]:
import os 
import pandas as pd
import glob
import pandas_profiling 
from functools import reduce

In [3]:
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

## Read data in the whole directory

### TODO: come back and teach batch size processing of data

In [4]:
file_path_list = glob.glob(os.path.join(data_folder, "*.csv"))

datadict = {os.path.splitext(os.path.basename(file))[0]: pd.read_csv(file) 
                for file in file_path_list}

for df_name, val in datadict.items():
    print(df_name, val.shape)

weather_description (45253, 37)
humidity (45253, 37)
wind_direction (45253, 37)
temperature (45253, 37)
pressure (45253, 37)
city_attributes (36, 4)
wind_speed (45253, 37)


In [37]:
type(datadict['weather_description'])

pandas.core.frame.DataFrame

In [46]:
datadict['city_attributes'].head()

,City,Country,Latitude,Longitude
0,Vancouver,Canada,49.249660,-123.119339
1,Portland,United States,45.523449,-122.676208
2,San Francisco,United States,37.774929,-122.419418
3,Seattle,United States,47.606209,-122.332069
4,Los Angeles,United States,34.052231,-118.243683


In [47]:
datadict['weather_description'].head()

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,haze,NaN,NaN,NaN
1,2012-10-01 13:00:00,mist,scattered clouds,light rain,sky is clear,mist,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,overcast clouds,sky is clear,sky is clear,sky is clear,haze,sky is clear,sky is clear,sky is clear
2,2012-10-01 14:00:00,broken clouds,scattered clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,sky is clear,few clouds,sky is clear,sky is clear,broken clouds,overcast clouds,sky is clear,overcast clouds
3,2012-10-01 15:00:00,broken clouds,scattered clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,sky is clear,few clouds,overcast clouds,sky is clear,broken clouds,overcast clouds,overcast clouds,overcast clouds
4,2012-10-01 16:00:00,broken clouds,scattered clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,sky is clear,few clouds,overcast clouds,sky is clear,broken clouds,overcast clouds,overcast clouds,overcast clouds


In [30]:
datadict['pressure'].sample(5)

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
26304,2015-10-02 12:00:00,1021.0,1023.0,1017.0,1023.0,1013.0,1013.0,1011.0,1009.0,1021.0,...,1021.0,1021.0,1032.0,1025.0,994.0,1016.0,1014.0,1017.0,1017.0,1016.0
19834,2015-01-05 22:00:00,1023.0,1040.0,1038.0,1037.0,1036.0,1035.0,1042.0,1036.0,1042.0,...,1040.0,1017.0,1024.0,1031.0,990.0,1015.0,958.0,1015.0,989.0,1026.0
30733,2016-04-04 01:00:00,1012.0,1015.0,1020.0,1014.0,1016.0,1017.0,1015.0,1014.0,1020.0,...,1016.0,1016.0,1016.0,1016.0,985.0,1013.0,969.0,1024.0,1024.0,1013.0
37220,2016-12-30 08:00:00,1021.0,1024.0,1025.0,1023.0,1019.0,1020.0,1024.0,1021.0,1030.0,...,1003.0,1002.0,993.0,993.0,991.0,1021.0,1021.0,1020.0,1020.0,1021.0
41161,2017-06-12 13:00:00,1015.0,1017.0,1011.0,1016.0,1016.0,1016.0,1009.0,1012.0,1018.0,...,1020.0,1018.0,1012.0,1023.0,982.0,1009.0,1006.0,1009.0,1009.0,1009.0


In [51]:
print("Minimum value: ", datadict['pressure'][['datetime']].min())
print("Maximum value: ", datadict["pressure"][['datetime']].max())

Minimum value:  datetime    2012-10-01 12:00:00
dtype: object
Maximum value:  datetime    2017-11-30 00:00:00
dtype: object


In [52]:
print("Minimum value: ", datadict['temperature'][['datetime']].min())
print("Maximum value: ", datadict["temperature"][['datetime']].max())

Minimum value:  datetime    2012-10-01 12:00:00
dtype: object
Maximum value:  datetime    2017-11-30 00:00:00
dtype: object


In [ ]:
humidity (45253, 37)
city_attributes (36, 4)

## Reshaping dataframe 

In [26]:
weather_desc_df = pd.melt(datadict['weather_description'], id_vars = ['datetime'], 
                          var_name='City', value_name='weather_descriptions')

pressure_df = pd.melt(datadict['pressure'], id_vars = ['datetime'], 
                          var_name='City', value_name='pressure')


wind_dir_df = pd.melt(datadict['wind_direction'], id_vars = ['datetime'], 
                          var_name='City', value_name='wind_direction')


wind_sp_df = pd.melt(datadict['wind_speed'], id_vars = ['datetime'], 
                          var_name='City', value_name='wind_speed')


temperature_df = pd.melt(datadict['temperature'], id_vars = ['datetime'], 
                          var_name='City', value_name='temperature')


humidity_df = pd.melt(datadict['humidity'], id_vars = ['datetime'], 
                          var_name='City', value_name='humidity')

print(pressure_df.shape)
print(wind_dir_df.shape)
print(wind_sp_df.shape)
print(temperature_df.shape)
print(humidity_df.shape)

(1629108, 3)
(1629108, 3)
(1629108, 3)
(1629108, 3)
(1629108, 3)


In [25]:
wind_dir_df.columns

Index(['datetime', 'city', 'wind_direction'], dtype='object')

## Merging dataframes 

### Basic "okay" way

We can do it one-by-one, like this:

df1 = pd.merge(pressure_df, wind_sp_df, on=['datetime', 'City'], how='outer')
df2 = pd.merge(df1, wind_dir_df, on=['datetime', 'City'], how='outer')
df3 = pd.merge(df2, ...)

### This is much more efficient

In [29]:
# compile the list of dataframes you want to merge
data_frames = [pressure_df, wind_sp_df, wind_dir_df, temperature_df, humidity_df, weather_desc_df]

# To keep the values that belong to the same date, you need to merge it on 'datetime'
df_weather_data = reduce(lambda left,right: pd.merge(left,right,on=['datetime', 'City'],
                                             how='outer'), data_frames)

df_merged = pd.merge(datadict['city_attributes'], df_weather_data, on='City', how='outer')

In [30]:
print(df_merged.shape)
df_merged.head()

(1629108, 11)


,City,Country,Latitude,Longitude,datetime,pressure,wind_speed,wind_direction,temperature,humidity,weather_descriptions
0,Vancouver,Canada,49.24966,-123.119339,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,Vancouver,Canada,49.24966,-123.119339,2012-10-01 13:00:00,NaN,0.0,0.0,284.630000,76.0,mist
2,Vancouver,Canada,49.24966,-123.119339,2012-10-01 14:00:00,NaN,0.0,6.0,284.629041,76.0,broken clouds
3,Vancouver,Canada,49.24966,-123.119339,2012-10-01 15:00:00,NaN,0.0,20.0,284.626998,76.0,broken clouds
4,Vancouver,Canada,49.24966,-123.119339,2012-10-01 16:00:00,NaN,0.0,34.0,284.624955,77.0,broken clouds


## Using pandas_profiling to get a simple overview of the data

In [32]:
profile = pandas_profiling.ProfileReport(df_merged)
# profile
profile.to_file(os.path.join(os.getcwd(), 'data_profile.html'))

### Looking at the data types, they all seem to be fine, except for 'datetime' column. It should be a date type

### Another handy tip for modifying column names in batches

In [31]:
keep_same = {'datetime'}
datadict['pressure'].columns = ['{}{}'.format(c, '' if c in keep_same else '_pressure') 
                                for c in datadict['pressure'].columns]
datadict['pressure'].columns

Index(['datetime', 'Vancouver_pressure', 'Portland_pressure',
       'San Francisco_pressure', 'Seattle_pressure', 'Los Angeles_pressure',
       'San Diego_pressure', 'Las Vegas_pressure', 'Phoenix_pressure',
       'Albuquerque_pressure', 'Denver_pressure', 'San Antonio_pressure',
       'Dallas_pressure', 'Houston_pressure', 'Kansas City_pressure',
       'Minneapolis_pressure', 'Saint Louis_pressure', 'Chicago_pressure',
       'Nashville_pressure', 'Indianapolis_pressure', 'Atlanta_pressure',
       'Detroit_pressure', 'Jacksonville_pressure', 'Charlotte_pressure',
       'Miami_pressure', 'Pittsburgh_pressure', 'Toronto_pressure',
       'Philadelphia_pressure', 'New York_pressure', 'Montreal_pressure',
       'Boston_pressure', 'Beersheba_pressure', 'Tel Aviv District_pressure',
       'Eilat_pressure', 'Haifa_pressure', 'Nahariyya_pressure',
       'Jerusalem_pressure'],
      dtype='object')